In [15]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as data
import datetime

russell_index = ["^RUI"]

portfolio = ["AMZN", "MSFT", "GOOG"]
assets = np.array([10000.0, 1000.0, 1000.0]) 
exposures = np.zeros(3)


#Set timeframe
start = "2015-12-07"
end = "2019-03-31"


#Pull data from Yahoo! Finance
df = data.DataReader(russell_index, data_source="yahoo", start=start, end=end)

#print(df["Adj Close"])

for i in enumerate(portfolio):
    df = data.DataReader(portfolio, data_source="yahoo", start=start, end=end)

    price = df["Adj Close"]
    print(price)
    # Assign the exposure at each index to the price times the allocation at each index
    #exposures[i] = price * assets[i] 
    # Parse data
    
    print(ticker+" Exposure in USD: ${:,.2f}".format(exposures[i]))
    







Symbols            AMZN         GOOG        MSFT
Date                                            
2015-12-07   669.830017   763.250000   51.467926
2015-12-08   677.330017   762.369995   51.449467
2015-12-09   664.789978   751.609985   50.702496
2015-12-10   662.320007   749.460022   50.969929
2015-12-11   640.150024   738.869995   49.854076
2015-12-14   657.909973   747.770020   50.850048
2015-12-15   658.640015   743.400024   50.905373
2015-12-16   675.770020   758.090027   51.763020
2015-12-17   670.650024   749.429993   51.366467
2015-12-18   664.140015   739.309998   49.918625
2015-12-21   664.510010   747.770020   50.564171
2015-12-22   663.150024   750.000000   51.043697
2015-12-23   663.700012   750.309998   51.477139
2015-12-24   662.789978   748.400024   51.338810
2015-12-28   675.200012   762.510010   51.597027
2015-12-29   693.969971   776.599976   52.150341
2015-12-30   689.070007   771.000000   51.929020
2015-12-31   675.890015   758.880005   51.163593
2016-01-04   636.989

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices